In [1]:
import hashlib
import time

class Block:
    def __init__(self, index, timestamp, file_hash, previous_hash):
        self.index = index
        self.timestamp = timestamp
        self.file_hash = file_hash
        self.previous_hash = previous_hash
        self.hash = self.compute_hash()

    def compute_hash(self):
        block_content = f"{self.index}{self.timestamp}{self.file_hash}{self.previous_hash}"
        return hashlib.sha256(block_content.encode()).hexdigest()


In [2]:
class Blockchain:
    def __init__(self):
        self.chain = []
        self.create_genesis_block()

    def create_genesis_block(self):
        genesis_block = Block(0, time.time(), "Genesis Block", "0")
        self.chain.append(genesis_block)

    def add_block(self, file_hash):
        last_block = self.chain[-1]
        new_block = Block(len(self.chain), time.time(), file_hash, last_block.hash)
        self.chain.append(new_block)

    def is_chain_valid(self):
        for i in range(1, len(self.chain)):
            current = self.chain[i]
            previous = self.chain[i - 1]
            if current.hash != current.compute_hash() or current.previous_hash != previous.hash:
                return False
        return True


In [3]:
import hashlib

def hash_file(file_path):
    hasher = hashlib.sha256()
    with open(file_path, 'rb') as file:
        while chunk := file.read(4096):
            hasher.update(chunk)
    return hasher.hexdigest()


In [4]:
import hashlib
import json
import sqlite3
from flask import Flask, request, jsonify # type: ignore
import os

# Initialize Flask app
app = Flask(__name__)

# Configure upload folder and database
UPLOAD_FOLDER = './uploads'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
DB_FILE = 'health_records.db'

# Initialize SQLite database
def init_db():
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS health_records (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            patient_data TEXT,
            file_path TEXT,
            block_hash TEXT,
            prev_hash TEXT
        )
    ''')
    conn.commit()
    conn.close()

init_db()

# Utility function to calculate hash
def calculate_hash(data):
    """Generate a SHA-256 hash for the given data."""
    data_string = json.dumps(data, sort_keys=True).encode()
    return hashlib.sha256(data_string).hexdigest()

@app.route('/add_health_record', methods=['POST'])
def add_health_record():
    patient_data = request.form.get('patient_data')
    if not patient_data:
        return jsonify({"message": "No patient data provided"}), 400

    file_path = None
    if 'file' in request.files:
        file = request.files['file']
        if file.filename != '':
            file_path = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
            file.save(file_path)

    # Fetch the last block to get its hash
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    cursor.execute("SELECT block_hash FROM health_records ORDER BY id DESC LIMIT 1")
    last_block = cursor.fetchone()
    prev_hash = last_block[0] if last_block else None

    # Create the new block
    new_block = {
        "patient_data": patient_data,
        "file_path": file_path,
        "prev_hash": prev_hash
    }
    block_hash = calculate_hash(new_block)

    # Save the new block to the database
    cursor.execute(
        "INSERT INTO health_records (patient_data, file_path, block_hash, prev_hash) VALUES (?, ?, ?, ?)",
        (patient_data, file_path, block_hash, prev_hash)
    )
    conn.commit()
    conn.close()

    return jsonify({"message": "Health record saved successfully", "block_hash": block_hash}), 201

@app.route('/verify_chain', methods=['GET'])
def verify_chain():
    """
    Verify the integrity of the blockchain by checking hashes.
    """
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM health_records ORDER BY id ASC")
    records = cursor.fetchall()
    conn.close()

    # Verify chain integrity
    prev_hash = None
    for record in records:
        block = {
            "patient_data": record[1],
            "file_path": record[2],
            "prev_hash": record[4]
        }
        calculated_hash = calculate_hash(block)
        if calculated_hash != record[3]:  # Compare calculated hash with stored hash
            return jsonify({"message": "Blockchain integrity compromised", "block_id": record[0]}), 400
        if record[4] != prev_hash:  # Verify previous hash matches
            return jsonify({"message": "Blockchain link broken", "block_id": record[0]}), 400
        prev_hash = record[3]

    return jsonify({"message": "Blockchain is valid"}), 200

if __name__ == '__main__':
    app.run(debug=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [27/Nov/2024 19:13:00] "POST /add_health_record HTTP/1.1" 201 -
127.0.0.1 - - [27/Nov/2024 19:13:07] "GET /verify_chain HTTP/1.1" 200 -
